# Grain Boundary Coupled Tilt Phase Field Model



In [ ]:
# -----------  Importing The Libraries ------------ # 
import numpy as np
import math as mt 
from sympy.vector import Del
from sympy import integrals as intp

In [ ]:
# -------------- Defining Parameters -------------- #

# Length of the simulation box
Ly = 1
# Normalized reference value
n0 = 1
# Chemical Potential at Equilibrium
muE = -1.33215 
# Chemical Potential of Solid Phase
muS = 0.9 * muE 
# Chemical Potential of Liquid Phase
muL = 1.1 * muE
# paramter used in evolution equation 
alpha = 0.1 
# Velocity of the crystals
v = 1.25 * mt.pow(10, -4)
# Parameter used in free energy density
epsilon = 0.25
# mesh - spacing in x direction
dx = 2 * np.pi / 8
# mesh - spacing in y direction
dy = 2 * np.pi / 8
# time-step
dt = 0.1
# length of unit box
a = 2 * np.pi 
# paramter used in Chemical Potential Function
b = 10 * np.pi
# Parameter used in Chemical Potential Function
xi = 4 * np.pi 
# Parameter used in G(y) Function
sigma = 2 * np.pi
# A value chosen such that
# the pulling is applied to solid strips 
# near the bottom and top surfaces that are not melted
d = 12 * a
# ratio of the magnitudes of reciprocal lattice vectors
Q1 = mt.sqrt(2)

## Chemical Potential Function 

### $\mu$ = $ \mu_{l} + \frac{(\mu_{l} - \mu_{s})}{2}(tanh[(y - L_{y} + b)/ \xi] - tanh[(y - b)/ \xi]) $

In [ ]:
# chemical potential function 
def mew(y):
    result1 = muL + ((muL - muS) / 2) * ((mt.tanh(y - Ly + 10)/xi) - (mt.tanh(y - b)/xi))
    return result1
    

## Crystal Field Density 

### $\psi(x,y,t) = \frac{(n(x,y,t) -  n_{0})}{n_{0}}$

In [ ]:
# crystal field density 
def Psi(x,y,t) :
    result2 = (n(x,y,t) - n0) / n0
    return result2

## Free Energy Density

### $f = \frac{\psi}{2} (-\epsilon +(\nabla^{2} + 1)^{2}[(\nabla ^{2} + Q_{1}^{2})])\psi + \frac{\psi^{4}}{4}$

In [ ]:
# free energy density 
def f(Psi) :
    result3 = (Psi / 2)(-epsilon + (Del()**2 + 1)*((Del()**2 + Q1**2)**2))*Psi + (Psi**4)/4
    return result3   

## External Force for Shearing 

### $ F_{ext} = \int dx dy [ G(y - d)(\psi (x,y,t) - \psi_{0}(x - vt,y))^{2} + G(y - L_{y} + d)(\psi (x,y,t) - \psi_{0}(x + vt,y))^{2}  ]$

In [1]:
# defining external force 
def Fext(x,y,t,G,Xi): 
    
    function = G(y - d)( Xi(x,y,t) - Xi(x - v*t,y,0) ) ** 2 + G(y - Ly + d)( Xi(x,y,t) - Xi(x + v*t,y,0) ) ** 2
    
    intwrty = intp.integrate(function,y) # integration wrt y
    
    intwrtx = intp.integrate(intwrty,x) # integration wrt x
    
    return intwrtx 

## G(y) function inside $F_{ext}$

### $ G(y) = exp(-y^{2}/2\sigma^{2}) / \sqrt {2 \pi \sigma^{2}} $

In [ ]:
def G(y):
    result4 = mt.exp(-(y*y)/2(sigma*sigma)) / mt.sqrt(2 * np.pi * (sigma*sigma))
    return result4 

## Free Energy Functional 

$F = \int drf  + F_{ext}  $

In [ ]:
# free energy functional 
def F(x,y,f,Fext) :
    result5 = intp.integrate(f,x) + intp.integrate(f,y) + Fext 
    return result5 